# Dispatch for `AbstractUnitcell` objects

Since a lot of information about unitcell objects is already encoded into their type signature, this signature can be used in dispatch to greatly reduce the number of `if` statements as well as greatly improve the performance of your code.

This tutorial aims at providing an overview how to use dispatch for `AbstractUnitcell` objects as well as giving some examples. It is very much advised to first work through the tutorials for [dispatching on site types ](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/sites_bonds/site_dispatch.ipynb) as well as [dispatching on bond types](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/sites_bonds/bond_dispatch.ipynb). Also, fundamental knowledge about how unitcells work should be present.

Since the requirements explain dispatch in greater detail, this tutorial will mainly focus on examples. Also note, that for the sake of simple examples, this tutorial will use pre-implemented unitcells from `LatPhysUnitcellLibrary`.

In [ ]:
using LatPhysBase
using LatPhysUnitcellLibrary

# Examples

Let's discuss some applications in the following



### Example 1 - bond offset vector from unitcell

If you worked through the tutorial on bond type dispatch, you will remember the example on calculating the offset vector for a given bond, given some lattice vectors. This example was very artificial since a lot of the information needed is not present in a bond object. However, it is available for unitcells, therefore the implementation of offsets should rather be conducted on the level of unitcells.

Read again throught the code of example 1 in the bond type dispatch [tutorial](https://github.com/janattig/LatticePhysics_Tutorials/blob/master/basics/sites_bonds/bond_dispatch.ipynb). The problems encountered there were the following:
1. site positions are not available from within a bond (only the site index)
2. lattice vectors are not available from within a bond
3. for the N=0 case, the spatial dimension is not known from within the bond

An implemntation on the level of unitcells is solving these three problems all at once:
1. site positions are known since the unitcell contains all sites
2. lattice vectors are known since the unitcell contains all lattice vectors
3. spatial dimension is known and even encoded into the unitcell type as the parameter `D` of the site type

In the following, we want to develope an offset function syntax to which it is only necessary to pass a bond and a unitcell object.



##### Fallback

Before any code is implemented, it should be worth thinking about what can go wrong and implement a fallback function for this case. The most obvious way to make a wrong function call in our case is to take a bond which is not fitting to the unitcell, e.g. because the bond expects a different number of lattice vectors than the unitcell is providing.

In [ ]:
# Fallback function (for any bond and any unitcell)
function offset(
        b :: B,
        u :: U
    ) where {N,L,B<:AbstractBond{L,N}, SU,BU,U<:AbstractUnitcell{SU,BU}}
   
    # give an error that the bond is not fitting to the unitcell
    @error "unitcell and bond are not fitting"
end

##### Fallback with fitting Bravais lattice dimension N

All relevant cases use the same Bravais lattice dimension `N` in both the passed bond, as well as the unitcell type. Generically, this could lead to the next level of fallback, e.g. creating a fallback function for using the same `N` but of a value that is not supported

In [ ]:
# Fallback function (for fitting bond and unitcell)
function offset(
        b :: B,
        u :: U
    ) where {N,L,B<:AbstractBond{L,N}, S,LU,BU<:AbstractBond{LU,N},U<:AbstractUnitcell{S,BU}}
    
    # give an error that the bond is not fitting to the unitcell
    @error "unitcell and bond are using unsupported N="*string(N)
end

##### Use cases for supported Bravais lattice dimension N

To show the above syntax in detail, let's consider only the case of N=2 in the following (this way, N=3 is unsupported). The following code provides a working offset calculation for a unitcell and bond of Bravais lattice dimensiion 2.

In [ ]:
# use case N=2
function offset(
        b :: B,
        u :: U
    ) :: Vector{Float64} where {L,B<:AbstractBond{L,2}, S,LU,BU<:AbstractBond{LU,2},U<:AbstractUnitcell{S,BU}}
    
    # return the offset vector
    return point(site(u,to(b))) .- point(site(u,from(b))) .+ wrap(b)[1].*a1(u) .+ wrap(b)[2].*a2(u)
end

##### Testing the implementation

Let us test the above syntax by providing three examples:
1. A working example for providing a bond from the unitcell that is passed
2. A working example by providing a made up bond compatible with the unitcell that is passed
3. An unsupported example by providing a bond from a unitcell with unsupported N
4. A broken example by providing unrelated bond and unitcell

In [ ]:
# Example 1 - A working example for providing a bond from the unitcell that is passed
uc = getUnitcellHoneycomb()
b  = bond(uc,2)
offset(b, uc)

In [ ]:
# Example 2 - A working example by providing a made up bond compatible with the unitcell that is passed
uc = getUnitcellHoneycomb()
b  = newBond( Bond{String,2}, 1,2, "mybond", (0,-2))
offset(b, uc)

In [ ]:
# Example 3 - An unsupported example by providing a bond from a unitcell with unsupported N
uc = getUnitcellCubic()
b  = bond(uc,2)
offset(b, uc)

In [ ]:
# Example 4 - A broken example by providing unrelated bond and unitcell
uc = getUnitcellCubic()
b  = newBond( Bond{String,2}, 1,2, "mybond", (0,-2))
offset(b, uc)